In [1]:
import pandas as pd
import os
import cv2
import numpy as np
import json
from skimage.transform import resize
from matplotlib import pyplot as plt
%matplotlib inline



In [2]:
with open("label_descriptions.json", "r")as f:
    labels_info = json.load(f)
# labels_info["categories"]

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()
unique_images = df.ImageId.unique()


In [5]:
df1 = df[:2]
df1

,ImageId,EncodedPixels,Height,Width,ClassId
0,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6
1,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0


In [6]:
def get_masks(image_id):
    masks = dict()
    temp = df[df.ImageId == image_id]
    for i in range(temp.shape[0]):
        width = temp.iloc[i].Width
        height = temp.iloc[i].Height
        class_id = temp.iloc[i].ClassId.split()[0]
        mask_encoded = temp.iloc[i].EncodedPixels.split()
        mask = [0] * (width*height)
        for j in range(0, len(mask_encoded), 2):
            mask[int(mask_encoded[j]): int(mask_encoded[j])+int(mask_encoded[j+1])] = [1]*int(mask_encoded[j+1])
        mask = np.fliplr(np.flip(np.rot90(np.array(mask).reshape((width, height)))))
        masks[int(class_id)] = mask
    masks_classes = []
    for i in range(46):
        if i in masks:
            new_mask = masks[i].astype('uint8') * i
            masks_classes.append(new_mask) 
    else:
        masks_classes.append(np.zeros((height, width)).astype('uint8'))
    masks_classes = np.array(masks_classes)
    return masks_classes

def join_masks(masks, path_to_annotation, img_name):
    all_masks = np.zeros((masks[0].shape[0],  masks[0].shape[1]))
    areas = {}
    for i, mask in enumerate(masks):
        areas[i] = np.count_nonzero(mask)
    areas_by_size = {k: v for k, v in sorted(areas.items(), key=lambda item: item[1])}
    for k, v in areas_by_size.items():
        all_masks = np.maximum(all_masks, masks[k])
    r = np.array(all_masks, dtype = np.uint8)
    cv2.imwrite(os.path.join(path_to_annotation, f"{img_name}.png"), r)


def create_annotation(path_to_annotation, img_id):
    img_name = unique_images[img_id].split(".")[0]
    masks = get_masks(unique_images[img_id])
    join_masks(masks, path_to_annotation, img_name)


In [7]:
create_annotation("train_annotations", 10)

In [8]:
for i, item in enumerate(unique_images[10:15]):
    create_annotation("val_annotations", i)

In [9]:
from shutil import copyfile
for file in os.listdir("val_annotations/"):
    filename = file.split('.')[0]
    copyfile(os.path.join("train", f"{filename}.jpg"), os.path.join("val_train", f"{filename}.jpg"))